In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as pl
import glob
from scipy.io.idl import readsav

In [ ]:
#pl.rcParams['figure.figsize'] = (12/2.54, 12/2.54)
pl.rcParams['font.size'] = 8
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.minor.visible'] = True
pl.rcParams['ytick.right'] = True
pl.rcParams['xtick.top'] = True
#pl.rcParams['figure.dpi'] = 96

In [ ]:
with open('../data_output/branch_points.json', 'r') as f:
    branch_points = json.load(f)

In [ ]:
models = list(branch_points['historical'].keys())
models

In [ ]:
piControls = {
    'ACCESS-CM2': 'r1i1p1f1',
    'ACCESS-ESM1-5': 'r1i1p1f1',
    'CAMS-CSM1-0': 'r1i1p1f1',
    'CanESM5': 'r1i1p1f1',
    'CESM2': 'r1i1p1f1',
    'CESM2-FV2': 'r1i1p1f1',
    'CESM2-WACCM': 'r1i1p1f1',
    'CESM2-WACCM-FV2': 'r1i1p1f1',
    'CIESM': 'r1i1p1f1',
    'CMCC-CM2-SR5': 'r1i1p1f1',
    'CMCC-ESM2': 'r1i1p1f1',
    'CNRM-CM6-1': 'r1i1p1f2',
    'CNRM-ESM2-1': 'r1i1p1f2',
    'E3SM-1-0': 'r1i1p1f1',
    'EC-Earth3': 'r1i1p1f1',
    'EC-Earth3-AerChem': 'r1i1p1f1',
    'FGOALS-g3': 'r1i1p1f1',
    'GFDL-CM4': 'r1i1p1f1',
    'GFDL-ESM4': 'r1i1p1f1',
    'GISS-E2-1-G': 'r1i1p1f2',
    'HadGEM3-GC31-LL': 'r1i1p1f1',
    'HadGEM3-GC31-MM': 'r1i1p1f1',
    'IPSL-CM5A2-INCA': 'r1i1p1f1',
    'IPSL-CM6A-LR': 'r1i1p1f1',
    'KACE-1-0-G': 'r1i1p1f1',
    'MIROC6': 'r1i1p1f1',
    'MIROC-ES2L': 'r1i1p1f2',
    'MPI-ESM-1-2-HAM': 'r1i1p1f1',
    'MPI-ESM1-2-HR': 'r1i1p1f1',
    'MRI-ESM2-0': 'r1i1p1f1',
    'NorESM2-LM': 'r1i1p1f1',
    'NorESM2-MM': 'r1i1p1f1',
    'TaiESM1': 'r1i1p1f1',
    'UKESM1-0-LL': 'r1i1p1f2',
}

In [ ]:
with open('../data_input/cmip56_forcing_feedback_ecs.json', 'rb') as f:
    feedbacks = json.load(f)

In [ ]:
lambda5 = {}
for model in feedbacks['CMIP5']:
    lambda5[model] = feedbacks['CMIP5'][model]['r1i1p1']['NET']
    
lambda6 = {}
for model in feedbacks['CMIP6']:
    if model not in piControls:
        continue
    fourtimes_run = piControls[model]
    if model=='EC-Earth3':  # no, I don't know why either
        fourtimes_run = 'r8i1p1f1'
    elif model=='GISS-E2-1-G':
        fourtimes_run = 'r1i1p1f1'
    elif model in ['HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
        fourtimes_run = 'r1i1p1f3'
    lambda6[model] = feedbacks['CMIP6'][model][fourtimes_run]['NET']

In [ ]:
piControl = {}
experiments = {}

In [ ]:
for model in models:
    piControl[model] = pd.read_csv('../data_output/cmip6/%s/%s/piControl.csv' % (model, piControls[model]))
piControl['CanESM5-cmip5'] = pd.read_csv('../data_output/cmip6/CanESM5/r1i1p1f1/piControl-cmip5.csv')

In [ ]:
piControl['CanESM5']

In [ ]:
piControl['CanESM5-cmip5']

In [ ]:
delta_N = {}
delta_T = {}
delta_F = {}

for experiment in ['historical', 'hist-GHG', 'hist-nat', 'hist-aer', 'historical-cmip5', 'hist-GHG-cmip5', 'hist-nat-cmip5', 'hist-aer-cmip5']:
    if experiment[-5:] == 'cmip5':
        nyears = 156
    else:
        nyears = 165
    delta_N[experiment] = {}
    delta_T[experiment] = {}
    delta_F[experiment] = {}
    for model in branch_points[experiment].keys():
        delta_N[experiment][model] = {}
        delta_T[experiment][model] = {}
        delta_F[experiment][model] = {}
        for run in list(branch_points[experiment][model].keys()):
            data = pd.read_csv('../data_output/cmip6/%s/%s/%s.csv' % (model, run, experiment))
            index_start = branch_points[experiment][model][run]
            N_historical = data['rsdt'].values[:nyears] - data['rsut'].values[:nyears] - data['rlut'].values[:nyears]
            if experiment[-5:] == 'cmip5':
                piC = piControl['CanESM5-cmip5']
            else:
                piC = piControl[model]
            N_piControl = (
                piC['rsdt'][index_start:index_start+nyears].values - 
                piC['rsut'][index_start:index_start+nyears].values - 
                piC['rlut'][index_start:index_start+nyears].values
            )
            delta_N[experiment][model][run] = N_historical - N_piControl
            delta_T[experiment][model][run] = data['tas'].values[:nyears] - piC['tas'][index_start:index_start+nyears].values
            delta_F[experiment][model][run] = delta_N[experiment][model][run] - lambda6[model] * delta_T[experiment][model][run]
        delta_F_array = np.ones((nyears, len(branch_points[experiment][model]))) * np.nan
        delta_N_array = np.ones((nyears, len(branch_points[experiment][model]))) * np.nan
        delta_T_array = np.ones((nyears, len(branch_points[experiment][model]))) * np.nan
        for i, run in enumerate(branch_points[experiment][model].keys()):
            delta_F_array[:, i] = delta_F[experiment][model][run]
            delta_N_array[:, i] = delta_N[experiment][model][run]
            delta_T_array[:, i] = delta_T[experiment][model][run]
        delta_F[experiment][model]['mean'] = np.mean(delta_F_array, axis=1)
        delta_N[experiment][model]['mean'] = np.mean(delta_N_array, axis=1)
        delta_T[experiment][model]['mean'] = np.mean(delta_T_array, axis=1)
        
        
delta_N['hist-otheranthro'] = {}
delta_T['hist-otheranthro'] = {}
delta_F['hist-otheranthro'] = {}
for model in branch_points['historical'].keys():
    if (model not in branch_points['hist-nat'].keys()) and (model not in branch_points['hist-GHG'].keys()):
        continue
    print(model)
    if (model in branch_points['hist-aer'].keys()):
        print(model)
    delta_N['hist-otheranthro'][model] = {}
    delta_T['hist-otheranthro'][model] = {}
    delta_F['hist-otheranthro'][model] = {}
    for run in list(branch_points['historical'][model].keys()):
        if (run not in branch_points['hist-nat'][model].keys()) or (run not in branch_points['hist-GHG'][model].keys()):
            continue
        delta_N['hist-otheranthro'][model][run] = delta_N['historical'][model][run] - delta_N['hist-GHG'][model][run] - delta_N['hist-nat'][model][run]
        delta_T['hist-otheranthro'][model][run] = delta_T['historical'][model][run] - delta_T['hist-GHG'][model][run] - delta_T['hist-nat'][model][run]
        delta_F['hist-otheranthro'][model][run] = delta_F['historical'][model][run] - delta_F['hist-GHG'][model][run] - delta_F['hist-nat'][model][run]
    delta_F_array = np.ones((165, len(delta_F['hist-otheranthro'][model]))) * np.nan
    delta_N_array = np.ones((165, len(delta_N['hist-otheranthro'][model]))) * np.nan
    delta_T_array = np.ones((165, len(delta_T['hist-otheranthro'][model]))) * np.nan
    for i, run in enumerate(delta_T['hist-otheranthro'][model].keys()):
        delta_F_array[:, i] = delta_F['hist-otheranthro'][model][run]
        delta_N_array[:, i] = delta_N['hist-otheranthro'][model][run]
        delta_T_array[:, i] = delta_T['hist-otheranthro'][model][run]
    delta_F['hist-otheranthro'][model]['mean'] = np.mean(delta_F_array, axis=1)
    delta_N['hist-otheranthro'][model]['mean'] = np.mean(delta_N_array, axis=1)
    delta_T['hist-otheranthro'][model]['mean'] = np.mean(delta_T_array, axis=1)

delta_N['hist-otheranthro-cmip5'] = {}
delta_T['hist-otheranthro-cmip5'] = {}
delta_F['hist-otheranthro-cmip5'] = {}
for model in branch_points['historical-cmip5'].keys():
    if (model not in branch_points['hist-nat-cmip5'].keys()) and (model not in branch_points['hist-GHG-cmip5'].keys()):
        continue
    delta_N['hist-otheranthro-cmip5'][model] = {}
    delta_T['hist-otheranthro-cmip5'][model] = {}
    delta_F['hist-otheranthro-cmip5'][model] = {}
    for run in list(branch_points['historical-cmip5'][model].keys()):
        if (run not in branch_points['hist-nat-cmip5'][model].keys()) or (run not in branch_points['hist-GHG-cmip5'][model].keys()):
            continue
        delta_N['hist-otheranthro-cmip5'][model][run] = delta_N['historical-cmip5'][model][run] - delta_N['hist-GHG-cmip5'][model][run] - delta_N['hist-nat-cmip5'][model][run]
        delta_T['hist-otheranthro-cmip5'][model][run] = delta_T['historical-cmip5'][model][run] - delta_T['hist-GHG-cmip5'][model][run] - delta_T['hist-nat-cmip5'][model][run]
        delta_F['hist-otheranthro-cmip5'][model][run] = delta_F['historical-cmip5'][model][run] - delta_F['hist-GHG-cmip5'][model][run] - delta_F['hist-nat-cmip5'][model][run]
    delta_F_array = np.ones((156, len(delta_F['hist-otheranthro-cmip5'][model]))) * np.nan
    delta_N_array = np.ones((156, len(delta_N['hist-otheranthro-cmip5'][model]))) * np.nan
    delta_T_array = np.ones((156, len(delta_T['hist-otheranthro-cmip5'][model]))) * np.nan
    for i, run in enumerate(delta_T['hist-otheranthro-cmip5'][model].keys()):
        delta_F_array[:, i] = delta_F['hist-otheranthro-cmip5'][model][run]
        delta_N_array[:, i] = delta_N['hist-otheranthro-cmip5'][model][run]
        delta_T_array[:, i] = delta_T['hist-otheranthro-cmip5'][model][run]
    delta_F['hist-otheranthro-cmip5'][model]['mean'] = np.mean(delta_F_array, axis=1)
    delta_N['hist-otheranthro-cmip5'][model]['mean'] = np.mean(delta_N_array, axis=1)
    delta_T['hist-otheranthro-cmip5'][model]['mean'] = np.mean(delta_T_array, axis=1)

In [ ]:
delta_F['hist-GHG']['MIROC6'].keys()

In [ ]:
run = 'r2i1p2f1'
model = 'CanESM5'
for run in list(branch_points['historical'][model].keys()):
    print (run, run not in branch_points['hist-nat'][model].keys()) and (run not in branch_points['hist-GHG'][model].keys())


In [ ]:
# grab the forcing
rfmip_erf = pd.read_csv('../data_input/RFMIP-ERF-tier2.csv')

In [ ]:
models_rfmip = ['CanESM5', 'CNRM-CM6-1', 'GFDL-CM4', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MIROC6', 'NorESM2-LM']

fig, ax = pl.subplots(4, 2, figsize=(16/2.54, 20/2.54))
for i, model in enumerate(models_rfmip):
    col = i%2
    row = i//2
    for run in branch_points['historical'][model].keys():
        if model=='CanESM5' and run=='r1i1p1f1':
            label='implied ERF ensemble members'
        else:
            label=''
        ax[row,col].plot(np.arange(1850.5, 2015), delta_F['historical'][model][run], color='0.7', label=label)
    ax[row,col].plot(np.arange(1850.5, 2015), delta_F['historical'][model]['mean'], color='k', label='implied ERF ensemble mean')
    ax[row,col].set_title('(%s) %s' % (chr(97+i), model))
    if '%s TOT' % model in rfmip_erf:
        ax[row,col].plot(np.arange(1850.5, 2015), rfmip_erf['%s TOT' % model].values[:165], color='b', label='RFMIP ERF')
    ax[row,col].set_xlim(1850, 2015)
    ax[row,col].set_ylim(-2, 3)
    ax[row,col].grid()
pl.figtext(0.01, 0.5, 'Historical effective radiative forcing (W m$^{-2}$)', rotation=90, va='center')
#ax[0,0].legend(loc='upper left', bbox_to_anchor=[0, -2.4], ncol=3)
ax[1,1].text(0.01, 0.88, 'implied ERF: ensemble members', transform=ax[1,1].transAxes, color='0.7', backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.1',ec='w',fc='w'))
ax[1,1].text(0.01, 0.76, 'implied ERF: ensemble mean', transform=ax[1,1].transAxes, backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.1',ec='w',fc='w'))
ax[1,1].text(0.01, 0.64, 'RFMIP ERF', transform=ax[1,1].transAxes, color='b', backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.1',ec='w',fc='w'))
fig.tight_layout(rect=[0.02, 0, 1, 1])
pl.savefig('../plots/fig1.pdf')

In [ ]:
fig, ax = pl.subplots(11, 4, figsize=(16,30))

for i, model in enumerate(models):
    col = i%4
    row = i//4
    for run in branch_points['historical'][model].keys():
        ax[row,col].plot(np.arange(1850.5, 2015), delta_F['historical'][model][run], color='0.7')
    ax[row,col].plot(np.arange(1850.5, 2015), delta_F['historical'][model]['mean'], color='k')
    ax[row,col].set_title(model)
    if '%s TOT' % model in rfmip_erf:
        ax[row,col].plot(np.arange(1850.5, 2015), rfmip_erf['%s TOT' % model].values[:165], color='r')
fig.tight_layout()

In [ ]:
fig, ax = pl.subplots(figsize=(10,10))
for model in models:
    ax.plot(np.arange(1850.5, 2015), delta_F['historical'][model]['mean'], label=model)
ax.legend()
ax.grid()
ax.set_ylabel('W m$^{-2}$');
ax.set_title('Derived historical ERF')
ax.set_xlim(1850,2015)
# compare 2014 with Smith et al

In [ ]:
fig, ax = pl.subplots(figsize=(10,10))
for model in models:
    ax.plot(np.arange(1850.5, 2015), delta_N['historical'][model]['mean'], label=model)
ax.legend()
ax.grid()
ax.set_ylabel('W m$^{-2}$');
ax.set_title('Top of atmosphere energy imbalance')
ax.set_xlim(1850,2015)

In [ ]:
fig, ax = pl.subplots(figsize=(10,10))
for model in models:
    ax.plot(np.arange(1850.5, 2015), delta_T['historical'][model]['mean'], label=model)
ax.legend()
ax.grid()
pl.ylabel('K');
pl.title('Temperature anomaly w.r.t parallel piControl')
ax.set_xlim(1850,2015)

In [ ]:
delta_F_cmip6_consolidated = {}
delta_N_cmip6_consolidated = {}
delta_T_cmip6_consolidated = {}
for experiment in ['historical', 'hist-nat', 'hist-GHG', 'hist-aer', 'hist-otheranthro']:
    delta_F_cmip6_consolidated[experiment] = np.zeros((165, len(delta_F[experiment])))
    delta_T_cmip6_consolidated[experiment] = np.zeros((165, len(delta_T[experiment])))
    delta_N_cmip6_consolidated[experiment] = np.zeros((165, len(delta_N[experiment])))
    for i, model in enumerate(delta_T[experiment].keys()):
        delta_F_cmip6_consolidated[experiment][:,i] = delta_F[experiment][model]['mean']
        delta_N_cmip6_consolidated[experiment][:,i] = delta_N[experiment][model]['mean']
        delta_T_cmip6_consolidated[experiment][:,i] = delta_T[experiment][model]['mean']

In [ ]:
delta_F['historical-cmip5']['CanESM5']

In [ ]:
# cmip5_df = pd.read_csv('../data_input/Forster2013_forcings_5yrsmooth_and_nosmooth.csv', skiprows=3, index_col=0, skipfooter=36)
# cmip5_df[cmip5_df==-99.99] = np.nan
# cmip5_df
# cmip5_df.mean(axis=1)

In [ ]:
hadcrut5 = pd.read_csv('../data_input/HadCRUT.5.0.1.0.analysis.summary_series.global.annual.csv', index_col=0)
hadcrut5.loc[:2020, 'Anomaly (deg C)'] - hadcrut5.loc[1850:1900,'Anomaly (deg C)'].mean()

In [ ]:
delta_N_cmip5 = {}
delta_T_cmip5 = {}
delta_F_cmip5 = {}

models = {}

for experiment in ['historical', 'historicalNat', 'historicalGHG']:
    delta_N_cmip5[experiment] = {}
    delta_T_cmip5[experiment] = {}
    delta_F_cmip5[experiment] = {}
    models[experiment] = set()
    for variable in ['rsdt', 'rsut', 'rlut', 'tas']:
        file_list = (glob.glob('/nfs/see-archive-15_a65/earpmf/CMIP5/cmip5/PROCESSED/%s_*_%s_*.idlsave' % (variable, experiment)))
        for file in file_list:
            models[experiment].add(file.split('_')[2])
            #print(variable, file.split('_')[2], file.split('_')[4][:-8])
    models[experiment] = (list(models[experiment]))
    for model in models[experiment]:
        if model not in lambda5:
            continue
        if model=='CCSM4':
            continue  # it's sick
        delta_N_cmip5[experiment][model] = {}
        delta_T_cmip5[experiment][model] = {}
        delta_F_cmip5[experiment][model] = {}
        file_list = glob.glob('/nfs/see-archive-15_a65/earpmf/CMIP5/cmip5/PROCESSED/rsdt_%s_%s_*.idlsave' % (model, experiment))
        for rsdtfile in file_list:
            run = rsdtfile.split('_')[4][:-8]
            tasfile = rsdtfile.replace('rsdt', 'tas')
            rsutfile = rsdtfile.replace('rsdt', 'rsut')
            rlutfile = rsdtfile.replace('rsdt', 'rlut')
                   
            try:
                tasdata = readsav(tasfile)
                len_tas = len(tasdata['tas'])
                rsdtdata = readsav(rsdtfile)
                len_rsdt = len(rsdtdata['rsdt'])
                rsutdata = readsav(rsutfile)
                len_rsut = len(rsutdata['rsut'])
                rlutdata = readsav(rlutfile)
                len_rlut = len(rlutdata['rlut'])
                tasdata['tas']-tasdata['ctrl'][:len_tas]
                rsdtdata['rsdt']-rsdtdata['ctrl'][:len_tas]
                rsutdata['rsut']-rsutdata['ctrl'][:len_tas]
                rlutdata['rlut']-rlutdata['ctrl'][:len_tas]
            except:
                continue
            
            delta_T_cmip5[experiment][model][run] = np.ones(156) * np.nan
            delta_N_cmip5[experiment][model][run] = np.ones(156) * np.nan
            delta_F_cmip5[experiment][model][run] = np.ones(156) * np.nan
            
            first_time = tasdata['time'][0]
            first_index = int(first_time - 1850)
            if model[:3] == 'Had':
                offset=-11
                first_index = first_index+1
            else:
                offset=0
            print(model, experiment, run, first_time, first_index)
            for i in range(first_index, 156):
                delta_T_cmip5[experiment][model][run][i] = np.mean(tasdata['tas'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(tasdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                rsdt = np.mean(rsdtdata['rsdt'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(rsdtdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                rsut = np.mean(rsutdata['rsut'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(rsutdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                rlut = np.mean(rlutdata['rlut'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(rlutdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                delta_N_cmip5[experiment][model][run][i] = rsdt - rsut - rlut
            delta_F_cmip5[experiment][model][run] = delta_N_cmip5[experiment][model][run] - lambda5[model] * delta_T_cmip5[experiment][model][run]
        delta_F_array = np.ones((nyears, len(delta_F_cmip5[experiment][model]))) * np.nan
        delta_N_array = np.ones((nyears, len(delta_N_cmip5[experiment][model]))) * np.nan
        delta_T_array = np.ones((nyears, len(delta_T_cmip5[experiment][model]))) * np.nan
        for i, run in enumerate(delta_F_cmip5[experiment][model].keys()):
            delta_F_array[:, i] = delta_F_cmip5[experiment][model][run]
            delta_N_array[:, i] = delta_N_cmip5[experiment][model][run]
            delta_T_array[:, i] = delta_T_cmip5[experiment][model][run]
        delta_F_cmip5[experiment][model]['mean'] = np.nanmean(delta_F_array, axis=1)
        delta_N_cmip5[experiment][model]['mean'] = np.nanmean(delta_N_array, axis=1)
        delta_T_cmip5[experiment][model]['mean'] = np.nanmean(delta_T_array, axis=1)

delta_N_cmip5['historicalOther'] = {}
delta_T_cmip5['historicalOther'] = {}
delta_F_cmip5['historicalOther'] = {}
for model in delta_T_cmip5['historical'].keys():
    if (model not in delta_T_cmip5['historicalNat'].keys()) and (model not in delta_T_cmip5['historicalGHG'].keys()):
        continue
    print(model)
    delta_N_cmip5['historicalOther'][model] = {}
    delta_T_cmip5['historicalOther'][model] = {}
    delta_F_cmip5['historicalOther'][model] = {}
    for run in list(delta_T_cmip5['historical'][model].keys()):
        if (run not in delta_T_cmip5['historicalGHG'][model].keys()) or (run not in delta_T_cmip5['historicalNat'][model].keys()):
            continue
        delta_N_cmip5['historicalOther'][model][run] = delta_N_cmip5['historical'][model][run] - delta_N_cmip5['historicalGHG'][model][run] - delta_N_cmip5['historicalNat'][model][run]
        delta_T_cmip5['historicalOther'][model][run] = delta_T_cmip5['historical'][model][run] - delta_T_cmip5['historicalGHG'][model][run] - delta_T_cmip5['historicalNat'][model][run]
        delta_F_cmip5['historicalOther'][model][run] = delta_F_cmip5['historical'][model][run] - delta_F_cmip5['historicalGHG'][model][run] - delta_F_cmip5['historicalNat'][model][run]
    delta_F_array = np.ones((156, len(delta_F_cmip5['historicalOther'][model]))) * np.nan
    delta_N_array = np.ones((156, len(delta_N_cmip5['historicalOther'][model]))) * np.nan
    delta_T_array = np.ones((156, len(delta_T_cmip5['historicalOther'][model]))) * np.nan
    for i, run in enumerate(delta_T_cmip5['historicalOther'][model].keys()):
        delta_F_array[:, i] = delta_F_cmip5['historicalOther'][model][run]
        delta_N_array[:, i] = delta_N_cmip5['historicalOther'][model][run]
        delta_T_array[:, i] = delta_T_cmip5['historicalOther'][model][run]
    delta_F_cmip5['historicalOther'][model]['mean'] = np.mean(delta_F_array, axis=1)
    delta_N_cmip5['historicalOther'][model]['mean'] = np.mean(delta_N_array, axis=1)
    delta_T_cmip5['historicalOther'][model]['mean'] = np.mean(delta_T_array, axis=1)

In [ ]:
delta_F_cmip5_consolidated = {}
delta_N_cmip5_consolidated = {}
delta_T_cmip5_consolidated = {}
for experiment in ['historical', 'historicalNat', 'historicalGHG', 'historicalOther']:
    delta_F_cmip5_consolidated[experiment] = np.zeros((156, len(delta_F_cmip5[experiment])))
    delta_T_cmip5_consolidated[experiment] = np.zeros((156, len(delta_T_cmip5[experiment])))
    delta_N_cmip5_consolidated[experiment] = np.zeros((156, len(delta_N_cmip5[experiment])))
    for i, model in enumerate(delta_T_cmip5[experiment].keys()):
        delta_F_cmip5_consolidated[experiment][:,i] = delta_F_cmip5[experiment][model]['mean']
        delta_N_cmip5_consolidated[experiment][:,i] = delta_N_cmip5[experiment][model]['mean']
        delta_T_cmip5_consolidated[experiment][:,i] = delta_T_cmip5[experiment][model]['mean']

In [ ]:
pl.plot(delta_F['historical']['CanESM5']['r11i1p1f1'])

In [ ]:
pl.plot(delta_F['historical']['CanESM5']['mean'])
pl.plot(delta_F['historical-cmip5']['CanESM5']['mean'])

In [ ]:
pl.plot(delta_F['hist-GHG']['CanESM5']['mean'])
pl.plot(delta_F['hist-GHG-cmip5']['CanESM5']['mean'])

In [ ]:
pl.plot(delta_F['hist-aer']['CanESM5']['mean'])
pl.plot(delta_F['hist-aer-cmip5']['CanESM5']['mean'])

In [ ]:
pl.plot(delta_F['hist-nat']['CanESM5']['mean'])
pl.plot(delta_F['hist-nat-cmip5']['CanESM5']['mean'])

In [ ]:
pl.plot(delta_T['hist-nat']['CanESM5']['mean'])
pl.plot(delta_T['hist-nat-cmip5']['CanESM5']['mean'])

In [ ]:
pl.plot(delta_T['historical']['CanESM5']['mean'])
pl.plot(delta_T['historical-cmip5']['CanESM5']['mean'])

In [ ]:
pl.plot(delta_T['hist-aer']['CanESM5']['mean'])
pl.plot(delta_T['hist-aer-cmip5']['CanESM5']['mean'])

In [ ]:
pl.plot(delta_T['hist-GHG']['CanESM5']['mean'])
pl.plot(delta_T['hist-GHG-cmip5']['CanESM5']['mean'])

In [ ]:
fig, ax = pl.subplots(3,1,figsize=(16/2.54, 20/2.54))

ax[0].fill_between(np.arange(1850.5,2015), np.mean(delta_T_cmip6_consolidated['hist-GHG'], axis=1)-np.std(delta_T_cmip6_consolidated['hist-GHG'], axis=1), np.mean(delta_T_cmip6_consolidated['hist-GHG'], axis=1)+np.std(delta_T_cmip6_consolidated['hist-GHG'], axis=1), color='blue', alpha=0.25)
ax[0].plot(np.arange(1850.5,2015), np.mean(delta_T_cmip6_consolidated['hist-GHG'], axis=1), label='CMIP6 (26 models and counting)', color='blue')
ax[0].fill_between(np.arange(1850.5,2006), np.nanmean(delta_T_cmip5_consolidated['historicalGHG'], axis=1)-np.nanstd(delta_T_cmip5_consolidated['historicalGHG'], axis=1), np.nanmean(delta_T_cmip5_consolidated['historicalGHG'], axis=1)+np.nanstd(delta_T_cmip5_consolidated['historicalGHG'], axis=1), color='red', alpha=0.25)
ax[0].plot(np.arange(1850.5,2006), np.nanmean(delta_T_cmip5_consolidated['historicalGHG'], axis=1), label='CMIP5 (Forster et al. 2013)', color='red')
ax[0].set_title('(a) GHG-induced warming ($\Delta T_{\mathrm{GHG}}$)')
ax[0].set_ylabel('K')
ax[0].grid()
#ax[0].legend()
ax[0].set_xlim(1850,2015)
ax[0].set_ylim(-0.2, 2)

ax[1].fill_between(np.arange(1850.5,2015), np.mean(delta_T_cmip6_consolidated['hist-nat'], axis=1)-np.std(delta_T_cmip6_consolidated['hist-nat'], axis=1), np.mean(delta_T_cmip6_consolidated['hist-nat'], axis=1)+np.std(delta_T_cmip6_consolidated['hist-nat'], axis=1), color='blue', alpha=0.25)
ax[1].plot(np.arange(1850.5,2015), np.mean(delta_T_cmip6_consolidated['hist-nat'], axis=1), label='CMIP6 (25 models and counting)', color='blue')
ax[1].fill_between(np.arange(1850.5,2006), np.nanmean(delta_T_cmip5_consolidated['historicalNat'], axis=1)-np.nanstd(delta_T_cmip5_consolidated['historicalNat'], axis=1), np.nanmean(delta_T_cmip5_consolidated['historicalNat'], axis=1)+np.nanstd(delta_T_cmip5_consolidated['historicalNat'], axis=1), color='red', alpha=0.25)
ax[1].plot(np.arange(1850.5,2006), np.nanmean(delta_T_cmip5_consolidated['historicalNat'], axis=1), label='CMIP5 (Forster et al. 2013)', color='red')
ax[1].set_title('(b) Natural-induced warming ($\Delta T_{\mathrm{nat}}$)')
ax[1].set_ylabel('K')
ax[1].grid()
#pl.legend()
ax[1].set_xlim(1850,2015)
ax[1].set_ylim(-0.5, 0.3)

ax[2].fill_between(np.arange(1850.5,2015), np.mean(delta_T_cmip6_consolidated['hist-otheranthro'], axis=1)-np.std(delta_T_cmip6_consolidated['hist-otheranthro'], axis=1), np.mean(delta_T_cmip6_consolidated['hist-otheranthro'], axis=1)+np.std(delta_T_cmip6_consolidated['hist-otheranthro'], axis=1), color='blue', alpha=0.25)
ax[2].plot(np.arange(1850.5,2015), np.mean(delta_T_cmip6_consolidated['hist-otheranthro'], axis=1), label='CMIP6 (10 models: non-GHG anthropogenic)', color='blue')
ax[2].fill_between(np.arange(1850.5,2015), np.mean(delta_T_cmip6_consolidated['hist-aer'], axis=1)-np.std(delta_T_cmip6_consolidated['hist-aer'], axis=1), np.mean(delta_T_cmip6_consolidated['hist-aer'], axis=1)+np.std(delta_T_cmip6_consolidated['hist-aer'], axis=1), color='green', alpha=0.25)
ax[2].plot(np.arange(1850.5,2015), np.mean(delta_T_cmip6_consolidated['hist-aer'], axis=1), label='CMIP6 (10 models: aerosol)', color='green')
ax[2].fill_between(np.arange(1850.5,2006), np.nanmean(delta_T_cmip5_consolidated['historicalOther'], axis=1)-np.nanstd(delta_T_cmip5_consolidated['historicalOther'], axis=1), np.nanmean(delta_T_cmip5_consolidated['historicalOther'], axis=1)+np.nanstd(delta_T_cmip5_consolidated['historicalOther'], axis=1), color='red', alpha=0.25)
ax[2].plot(np.arange(1850.5,2006), np.nanmean(delta_T_cmip5_consolidated['historicalOther'], axis=1), label='CMIP5 (Forster et al. 2013)', color='red')
ax[2].set_title('(c) Other anthropogenic-induced warming ($\Delta T_{\mathrm{other}}$)')
ax[2].set_ylabel('K')
ax[2].grid()
ax[2].legend()
ax[2].set_xlim(1850,2015)
ax[2].set_ylim(-1.2,0.3)

fig.tight_layout()

In [ ]:
lambda5

In [ ]:
delta_F['hist-aer-cmip5']['CanESM5']

In [ ]:
nmodels = {}
nmodels['CMIP5'] = {}
for expt in ['historical','historicalGHG','historicalNat','historicalOther']:
    count = 0
    for row in delta_F_cmip5_consolidated[expt].T:
        count = count + (1-np.all(np.isnan(row)))
    nmodels['CMIP5'][expt]=count
nmodels['CMIP6'] = {}
for expt in ['historical','hist-GHG','hist-nat','hist-aer','hist-otheranthro']:
    count = 0
    for row in delta_F_cmip6_consolidated[expt].T:
        count = count + (1-np.all(np.isnan(row)))
    nmodels['CMIP6'][expt]=count

In [ ]:
nmodels

In [ ]:
goodmodels = {}
sens = {}
goodmodels['CMIP5'] = {}
sens['CMIP5'] = {}
for expt in ['historical','historicalGHG','historicalNat','historicalOther']:
    goodlist = []
    senslist = []
    for model in delta_F_cmip5[expt]:
        if not (np.all(np.isnan(delta_F_cmip5[expt][model]['mean']))):
            goodlist.append(model)
            senslist.append(-1 / lambda5[model])
    goodmodels['CMIP5'][expt]=goodlist
    sens['CMIP5'][expt] = np.array(senslist)
goodmodels['CMIP6'] = {}
sens['CMIP6'] = {}
for expt in ['historical','hist-GHG','hist-nat','hist-aer','hist-otheranthro']:
    goodlist = []
    senslist = []
    for model in delta_F[expt]:
        if not (np.all(np.isnan(delta_F[expt][model]['mean']))):
            goodlist.append(model)
            senslist.append(-1 / lambda6[model])
    goodmodels['CMIP6'][expt]=goodlist
    sens['CMIP6'][expt] = np.array(senslist)

In [ ]:
for expt in ['historical','hist-GHG','hist-nat','hist-aer','hist-otheranthro']:
    print(expt, sens['CMIP6'][expt].mean(), sens['CMIP6'][expt].std())
for expt in ['historical','historicalGHG','historicalNat','historicalOther']:
    print(expt, sens['CMIP5'][expt].mean(), sens['CMIP5'][expt].std())

In [ ]:
fig, ax = pl.subplots(4,2,figsize=(16/2.54, 20/2.54))

cmip5_names = {
    'historical': 'historical',
    'hist-GHG': 'historicalGHG',
    'hist-nat': 'historicalNat',
    'hist-otheranthro': 'historicalOther',
}

for i, expt in enumerate(['historical', 'hist-GHG','hist-nat','hist-otheranthro']):
    ax[i,0].fill_between(
        np.arange(1850.5,2015), 
        np.mean(delta_F_cmip6_consolidated[expt], axis=1)-np.std(delta_F_cmip6_consolidated[expt], axis=1),
        np.mean(delta_F_cmip6_consolidated[expt], axis=1)+np.std(delta_F_cmip6_consolidated[expt], axis=1),
        color='blue',
        alpha=0.2,
        edgecolor=None
    )
    ax[i,0].plot(
        np.arange(1850.5,2015),
        np.mean(delta_F_cmip6_consolidated[expt], axis=1),
        label='CMIP6 (%d)' % (nmodels['CMIP6'][expt]),
        color='blue'
    )
    ax[i,0].fill_between(
        np.arange(1850.5,2006),
        np.nanmean(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1)-np.nanstd(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1),
        np.nanmean(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1)+np.nanstd(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1),
        color='red',
        alpha=0.2,
        edgecolor=None
    )
    ax[i,0].plot(
        np.arange(1850.5,2006),
        np.nanmean(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1),
        label='CMIP5',
        color='red'
    )
    if i==3:
        ax[3,0].plot(np.arange(1850.5,2015), np.mean(delta_F_cmip6_consolidated['hist-aer'], axis=1), color='darkblue', lw=1.0)
    ax[i,0].set_ylabel('W m$^{-2}$')
    ax[i,0].grid()
    ax[i,0].set_title(expt)
    ax[i,0].set_xlim(1850, 2015)
    #ax[row,col].set_ylim(-2, 3)

    ax[i,1].fill_between(
        np.arange(1850.5,2015), 
        np.mean(delta_T_cmip6_consolidated[expt], axis=1)-np.std(delta_T_cmip6_consolidated[expt], axis=1),
        np.mean(delta_T_cmip6_consolidated[expt], axis=1)+np.std(delta_T_cmip6_consolidated[expt], axis=1),
        color='blue',
        alpha=0.2,
        edgecolor=None
    )
    ax[i,1].plot(
        np.arange(1850.5,2015),
        np.mean(delta_T_cmip6_consolidated[expt], axis=1),
        label='CMIP6 (%d)' % (nmodels['CMIP6'][expt]),
        color='blue'
    )
    ax[i,1].fill_between(
        np.arange(1850.5,2006),
        np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1)-np.nanstd(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
        np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1)+np.nanstd(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
        color='red',
        alpha=0.2,
        edgecolor=None
    )
    ax[i,1].plot(
        np.arange(1850.5,2006),
        np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
        label='CMIP5 (%d)' % (nmodels['CMIP5'][cmip5_names[expt]]),
        color='red'
    )
    if i==0:
        ax[0,1].plot(
            hadcrut5.loc[:2020, 'Anomaly (deg C)'] - hadcrut5.loc[1850:1900,'Anomaly (deg C)'].mean(),
            color='k', 
            label='HadCRUT5',
            lw=1.0
        )
    if i==3:
        ax[3,1].plot(
            np.arange(1850.5,2015),
            np.mean(delta_T_cmip6_consolidated['hist-aer'], axis=1),
            label='CMIP6 aerosol-only (%d)' % (nmodels['CMIP6']['hist-aer']),
            color='darkblue',
            lw=1.0,
        )

    ax[i,1].set_ylabel('$^{\circ}$C')
    ax[i,1].grid()
    ax[i,1].set_title(expt)
    ax[i,1].set_xlim(1850, 2015)
    ax[i,1].legend()

ax[0,0].set_ylim(-2,2.5)
ax[0,1].set_ylim(-0.4,1.4)
ax[1,0].set_ylim(-1,3.2)
ax[1,1].set_ylim(-0.2,2)
ax[2,0].set_ylim(-3,1)
ax[2,1].set_ylim(-0.5,0.25)
ax[3,0].set_ylim(-2,1)
ax[3,1].set_ylim(-1.05,0.3)

ax[0,0].set_title('(a) Implied total ERF ($\Delta F$)')
ax[1,0].set_title('(c) Implied greenhouse gas ERF ($\Delta F_{\mathrm{GHG}}$)')
ax[2,0].set_title('(e) Implied natural ERF ($\Delta F_{\mathrm{nat}}$)')
ax[3,0].set_title('(g) Implied other anthropogenic ERF ($\Delta F_{\mathrm{other}}$)')
ax[0,1].set_title('(b) Total warming ($\Delta T$)')
ax[1,1].set_title('(d) Greenhouse gas warming ($\Delta T_{\mathrm{GHG}}$)')
ax[2,1].set_title('(f) Natural warming ($\Delta T_{\mathrm{nat}}$)')
ax[3,1].set_title('(h) Other anthropogenic warming ($\Delta T_{\mathrm{other}}$)')
fig.tight_layout()
pl.savefig('../plots/fig2.pdf')

In [ ]:
fig, ax = pl.subplots(4,2,figsize=(16/2.54, 20/2.54))

for i, expt in enumerate(['historical', 'hist-GHG','hist-nat','hist-otheranthro']):
    ax[i,0].plot(
        np.arange(1850.5,2015),
        delta_F[expt]['CanESM5']['mean'],
        label='CMIP6 (%d)' % (nmodels['CMIP6'][expt]),
        color='blue'
    )
    ax[i,0].plot(
        np.arange(1850.5,2006),
        delta_F[expt + '-cmip5']['CanESM5']['mean'],
        label='CMIP5',
        color='red'
    )
    ax[i,0].set_ylabel('W m$^{-2}$')
    ax[i,0].grid()
    ax[i,0].set_title(expt)
    ax[i,0].set_xlim(1850, 2015)
    #ax[row,col].set_ylim(-2, 3)

    if i==0:
        label = 'CanESM5, CMIP6 forcing'
    else:
        label = ''
    ax[i,1].plot(
        np.arange(1850.5,2015),
        delta_T[expt]['CanESM5']['mean'],
        label=label,
        color='blue'
    )
    if i==0:
        label = 'CanESM5, CMIP5 forcing'
    else:
        label = ''
    ax[i,1].plot(
        np.arange(1850.5,2006),
        delta_T[expt + '-cmip5']['CanESM5']['mean'],
        label=label,
        color='red'
    )
    if i==0:
        ax[0,1].plot(
            hadcrut5.loc[:2020, 'Anomaly (deg C)'] - hadcrut5.loc[1850:1900,'Anomaly (deg C)'].mean(),
            color='k', 
            label='HadCRUT5',
            lw=1.0
        )
        ax[i,1].legend()

    ax[i,1].set_ylabel('$^{\circ}$C')
    ax[i,1].grid()
    ax[i,1].set_title(expt)
    ax[i,1].set_xlim(1850, 2015)


ax[0,0].set_ylim(-2,2.5)
ax[0,1].set_ylim(-0.5,1.5)
ax[1,0].set_ylim(-0.4,2.8)
ax[1,1].set_ylim(-0.2,2.4)
ax[2,0].set_ylim(-3,1)
ax[2,1].set_ylim(-0.6,0.25)
ax[3,0].set_ylim(-1.6,0.5)
ax[3,1].set_ylim(-1.1,0.3)

ax[0,0].set_title('(a) Implied total ERF ($\Delta F$)')
ax[1,0].set_title('(c) Implied greenhouse gas ERF ($\Delta F_{\mathrm{GHG}}$)')
ax[2,0].set_title('(e) Implied natural ERF ($\Delta F_{\mathrm{nat}}$)')
ax[3,0].set_title('(g) Implied aerosol ERF ($\Delta F_{\mathrm{aer}}$)')
ax[0,1].set_title('(b) Total warming ($\Delta T$)')
ax[1,1].set_title('(d) Greenhouse gas warming ($\Delta T_{\mathrm{GHG}}$)')
ax[2,1].set_title('(f) Natural warming ($\Delta T_{\mathrm{nat}}$)')
ax[3,1].set_title('(h) Aerosol warming ($\Delta T_{\mathrm{aer}}$)')
fig.tight_layout()
pl.savefig('../plots/fig3.pdf')

In [ ]:
for model in delta_F_cmip5['historicalOther']:
    pl.plot(np.arange(1850.5,2006), delta_F_cmip5['historicalOther'][model]['mean'], label=model)
pl.legend()

In [ ]:
np.nanmean(delta_T_cmip5_consolidated['historical'], axis=1)[:51].mean()

In [ ]:
np.nanmean(delta_T_cmip6_consolidated['historical'], axis=1)[:51].mean()

In [ ]:
np.mean(np.mean(delta_T_cmip6_consolidated['hist-aer'][145:156,:], axis=0))

In [ ]:
np.std(np.mean(delta_T_cmip6_consolidated['hist-aer'][145:156,:], axis=0))

In [ ]:
for expt in ['historical', 'hist-GHG','hist-nat','hist-otheranthro','hist-aer']:
    print('%16s & $%.2f \pm %.2f$ & $%.2f \pm %.2f$ & $%.2f$' % 
        (
            expt,
            np.mean(np.mean(delta_F_cmip6_consolidated[expt][145:156,:], axis=0)),
            np.std(np.mean(delta_F_cmip6_consolidated[expt][145:156,:], axis=0)),
            np.mean(np.mean(delta_T_cmip6_consolidated[expt][145:156,:], axis=0)),
            np.std(np.mean(delta_T_cmip6_consolidated[expt][145:156,:], axis=0)),
            np.mean(np.mean(delta_T_cmip6_consolidated[expt][145:156,:], axis=0)) / np.mean(np.mean(delta_F_cmip6_consolidated[expt][145:156,:], axis=0))
        )
    )

In [ ]:
for expt in ['historical', 'historicalGHG','historicalNat','historicalOther']:
    print('$%.2f \pm %.2f$ & $%.2f \pm %.2f$ & $%.2f$ \\\\' % 
        (
            np.nanmean(np.nanmean(delta_F_cmip5_consolidated[expt][145:156,:], axis=0)),
            np.nanstd(np.nanmean(delta_F_cmip5_consolidated[expt][145:156,:], axis=0)),
            np.nanmean(np.nanmean(delta_T_cmip5_consolidated[expt][145:156,:], axis=0)),
            np.nanstd(np.nanmean(delta_T_cmip5_consolidated[expt][145:156,:], axis=0)),
            np.nanmean(np.nanmean(delta_T_cmip5_consolidated[expt][145:156,:], axis=0)) / np.nanmean(np.nanmean(delta_F_cmip5_consolidated[expt][145:156,:], axis=0))
        )
    )

In [ ]:
delta_T_cmip6_consolidated['hist-nat'].mean(axis=1)[145:156]